<h1 align=center><font size = 5>Opening New Bar in Delhi,India</font></h1>

## Introduction

The aim of the project is to identify best neighbourhoods to open bar in Delhi, India based on the distribution of the bars in the neighbourhoods of Delhi which we will explore using Latitude and Longitude values. In this notebook, we will explore the city of Delhi, India, using Foursquare API,to help others to decide where should they open bar that will be possibly the best location.

As Delhi is a vast city it's not easy to take decision where to open bar so that it can target more number of people and hence maximize the profit.

Here, we'll identify places that are fit for this purpose of opening bar based on the information collected from the Foresquare API and Data Science. Once we have this complete data about venues in these neighbourhoods in Delhi, any person can easily decide the best neighbourhoods for this purpose.

## Table of Contents

    
1. Scrap Wekipedia Page and Building DataFrame
    
2. Get Geographical Cordinates of Neighbourhoods

3. Explore Neighborhoods in Delhi

4. Analyze Each Neighborhoods

5. Clusters Neighbourhoods  

6. Examine Clusters 

7. Identifying cluster of Interest


In [1]:
#Importing necessary libraries


import requests
from bs4 import BeautifulSoup
import json 
import numpy as np

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

import folium 

print('Libraries imported.')

Libraries imported.


<h3 id="item1">1. Scrap Wekipedia Page and Building DataFrame</h3>

#### Reading data from wikipedia

In [2]:
url='https://en.wikipedia.org/wiki/Neighbourhoods_of_Delhi'
data=requests.get(url).text
data

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Neighbourhoods of Delhi - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"9d61dd89-bbdf-4ca4-a056-ffb332f62719","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Neighbourhoods_of_Delhi","wgTitle":"Neighbourhoods of Delhi","wgCurRevisionId":973055826,"wgRevisionId":973055826,"wgArticleId":7375737,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Use dmy dates from December 2018","Use Indian English from January 2016","All Wikipedia articles written in Indian English","Commons 

#### Using BeautifulSoup to parse data

In [3]:
soup = BeautifulSoup(data,'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Neighbourhoods of Delhi - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"9d61dd89-bbdf-4ca4-a056-ffb332f62719","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Neighbourhoods_of_Delhi","wgTitle":"Neighbourhoods of Delhi","wgCurRevisionId":973055826,"wgRevisionId":973055826,"wgArticleId":7375737,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Use dmy dates from December 2018","Use Indian English from January 2016","All Wikipedia articles written in Indian Englis

#### Getting List element from HTML text 

In [4]:
list_Element=soup.find_all('li')

boroughs=[]
for b in list_Element[0:9]: #first 9 elements of list_Element elements are borough name
    boroughs.append(b.text[1:])   


df_br=pd.DataFrame({'Borough':boroughs})
df_br['NeighbourhoodCount']=[9,13,6,7,7,10,37,15,13] #Inspection of counts of neighbourhoods in each borough in wiki page 
df_br.head()

Borough  NeighbourhoodCount
0   North West Delhi                   9
1        North Delhi                  13
2   North East Delhi                   6
3      Central Delhi                   7
4          New Delhi                   7

#### Creating dataframe containing boroughs and neighbourhoods

In [5]:
#Preparing Neighbourhood list
Neighbourhood=list_Element[12:] #Neighbourhoods are after 11th position in list_Element

In [6]:
count=0
boroughList=[]
NeighbourhoodList=[]
for borough,total in zip(df_br['Borough'],df_br['NeighbourhoodCount']):
    for _ in range(total):
        boroughList.append(borough)        
        NeighbourhoodList.append(Neighbourhood[count].text)
        count+=1

#Creating dataframe with boroughs and neighbourhoods    
df=pd.DataFrame({'Borough':boroughList,'Neighbourhood':NeighbourhoodList})

In [7]:
df.head()

Borough Neighbourhood
0   North West Delhi  Adarsh Nagar
1   North West Delhi   Ashok Vihar
2   North West Delhi     Begum Pur
3   North West Delhi        Karala
4   North West Delhi    Model Town

#### Data correction for finding geographical co-ordinates

In [8]:
df.loc[df['Neighbourhood'] =='Sundar Nagar[1]', 'Neighbourhood'] = 'Sundar Nagar'
df.loc[df['Neighbourhood'] =='Begum Pur', 'Neighbourhood'] = 'BegumPur'
df.loc[df['Neighbourhood'] =='Rohini Sub City', 'Neighbourhood'] = 'Rohini'
df.loc[df['Neighbourhood'] =='Rajender Nagar', 'Neighbourhood'] = 'Rajinder Nagar'
df.loc[df['Neighbourhood'] =='Dwarka Sub City', 'Neighbourhood'] = 'Dwarka'
df.loc[df['Neighbourhood'] =='Sagar Pur', 'Neighbourhood'] = 'SagarPur'

In [9]:
#Displaying DataFrame
df.head()

Borough Neighbourhood
0   North West Delhi  Adarsh Nagar
1   North West Delhi   Ashok Vihar
2   North West Delhi      BegumPur
3   North West Delhi        Karala
4   North West Delhi    Model Town

## 2. Get Geographical Cordinates of Neighbourhoods

In [10]:
latitudes=[]
longitudes=[]
for neighbourhood in df['Neighbourhood']:
    address = '{},{}'.format(neighbourhood,'Delhi')
    
    geolocator = Nominatim(user_agent="dl_explorer")    
    location = geolocator.geocode(address)
    latitudes.append(location.latitude)
    longitudes.append(location.longitude)    

#### Adding two new columns containing Latitude and Longitude values of neighbourhoods to existing dataframe

In [11]:
df['Latitude']=latitudes
df['Longitude']=longitudes
df.head()

Borough Neighbourhood   Latitude  Longitude
0   North West Delhi  Adarsh Nagar  28.614193  77.071541
1   North West Delhi   Ashok Vihar  28.699453  77.184826
2   North West Delhi      BegumPur  28.536978  77.209941
3   North West Delhi        Karala  28.735140  77.032511
4   North West Delhi    Model Town  28.702714  77.193991

#### Saving final dataframe containing Neighbourhoods with geographical coordinates in csv format

In [12]:
df.to_csv('Delhi_Geographical_Cordinates.csv')

#### Calculating Total Borough and Neighbourhoods

In [13]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 9 boroughs and 117 neighborhoods.


#### Finding latitude and longitude of Delhi,the capital of India

In [14]:
address = 'Delhi,India'

geolocator = Nominatim(user_agent="dl_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Delhi are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Delhi are 28.6517178, 77.2219388.


#### Creating map of Delhi and superimposing its neighbourhoods on it

In [15]:
map_delhi = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_delhi)  
    
map_delhi

#### Saving map of delhi

In [16]:
map_delhi.save('map_delhi.html')

## 3. Explore Neighborhoods in Delhi

#### Defining Foursquare credentials

In [17]:
CLIENT_ID = 'XB211O2DGNOE20ZFWAC1MT4U1HZNOLRJ4L3LL1Y03NAISJRZ'
CLIENT_SECRET = '2UOVL44N5AKKEVNXJAB0A01EITNSZWWGC1CZUJ051CK2HNHT' 
VERSION = '20200814'
radius=2000
LIMIT=100

print('Foresquare credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Foresquare credentails:
CLIENT_ID: XB211O2DGNOE20ZFWAC1MT4U1HZNOLRJ4L3LL1Y03NAISJRZ
CLIENT_SECRET:2UOVL44N5AKKEVNXJAB0A01EITNSZWWGC1CZUJ051CK2HNHT


#### Getting venues in Neighbourhoods of Delhi

In [18]:
venues_list=[]
for name, lat, lng in zip(df["Neighbourhood"],df['Latitude'],df['Longitude']):

    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)

    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']

    # return only relevant information for each nearby venue
    venues_list.append([(
        name, 
        lat, 
        lng, 
        v['venue']['name'], 
        v['venue']['location']['lat'], 
        v['venue']['location']['lng'],  
        v['venue']['categories'][0]['name']) for v in results])

nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['Neighborhood', 
              'Neighborhood Latitude', 
              'Neighborhood Longitude', 
              'Venue', 
              'Venue Latitude', 
              'Venue Longitude', 
              'Venue Category']


#### Printing Shape of this dataframe and 5 records

In [19]:
print(nearby_venues.shape)
nearby_venues.head()

(5160, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Adarsh Nagar              28.614193               77.071541   
1  Adarsh Nagar              28.614193               77.071541   
2  Adarsh Nagar              28.614193               77.071541   
3  Adarsh Nagar              28.614193               77.071541   
4  Adarsh Nagar              28.614193               77.071541   

             Venue  Venue Latitude  Venue Longitude     Venue Category  
0    Shree Maakhan       28.625825        77.068608  Indian Restaurant  
1  Cafe Coffee Day       28.622388        77.088353               Café  
2   Domino's Pizza       28.630000        77.080000        Pizza Place  
3  Cafe Coffee Day       28.629794        77.079771        Coffee Shop  
4           Subway       28.630026        77.079541     Sandwich Place

#### Total number of venues in each Neighbourhood

In [20]:
nearby_venues.groupby('Neighborhood').count()

Neighborhood Latitude  Neighborhood Longitude  Venue  \
Neighborhood                                                               
Adarsh Nagar                           11                      11     11   
Alaknanda                              58                      58     58   
Anand Vihar                            38                      38     38   
Ashok Nagar                            49                      49     49   
Ashok Vihar                            29                      29     29   
Badarpur                               11                      11     11   
Bali Nagar                             48                      48     48   
Barakhamba Road                       100                     100    100   
BegumPur                              100                     100    100   
Chanakyapuri                           72                      72     72   
Chandni Chowk                          56                      56     56   
Chittaranjan Park                      89                      89     89   
Civil Lines                            33                      33     33   
Connaught Place                       100                     100    100   
Dabri, New Delhi                        7                       7      7   
Daryaganj                              32                      32     32   
Defence Colony                         94                      94     94   
Delhi Cantonment                       11                      11     11   
Dhaula Kuan                            43                      43     43   
Dilshad Garden                         10                      10     10   
Dwarka                                 14                      14     14   
East Vinod Nagar                       11                      11     11   
East of Kailash                       100                     100    100   
Fateh Nagar                            46                      46     46   
Gole Market                           100                     100    100   
Govindpuri                             34                      34     34   
Greater Kailash                       100                     100    100   
Green Park                            100                     100    100   
Gulabi Bagh                            32                      32     32   
Gulmohar Park                         100                     100    100   
Hauz Khas                             100                     100    100   
Hauz Khas Village                     100                     100    100   
INA Colony                             68                      68     68   
Inderpuri                              11                      11     11   
Jangpura                               87                      87     87   
Jhandewalan                            68                      68     68   
Jhilmil Colony                         15                      15     15   
Jor Bagh                               28                      28     28   
Kalkaji                                70                      70     70   
Kamla Nagar                            34                      34     34   
Karala                                  2                       2      2   
Karol Bagh                             41                      41     41   
Kashmiri Gate                          35                      35     35   
Khanpur                                 8                       8      8   
Kirti Nagar                            36                      36     36   
Kishanganj                             23                      23     23   
Kotwali                                86                      86     86   
Lajpat Nagar                           92                      92     92   
Laxmi Nagar                            31                      31     31   
Laxmibai Nagar                         69                      69     69   
Lodi Colony                            72                      72   

#### Let's find out how many unique categories can be curated from all the returned venues

In [21]:
print('There are {} uniques categories.'.format(len(nearby_venues['Venue Category'].unique())))

There are 212 uniques categories.


## 4. Analyze Each Neighborhoods

In [22]:
# one hot encoding
nearby_onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nearby_onehot['Neighbourhood'] = nearby_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [nearby_onehot.columns[-1]] + list(nearby_onehot.columns[:-1])
nearby_onehot = nearby_onehot[fixed_columns]

nearby_onehot.head()

Neighbourhood  ATM  Accessories Store  Airport  Airport Food Court  \
0  Adarsh Nagar    0                  0        0                   0   
1  Adarsh Nagar    0                  0        0                   0   
2  Adarsh Nagar    0                  0        0                   0   
3  Adarsh Nagar    0                  0        0                   0   
4  Adarsh Nagar    0                  0        0                   0   

   Airport Service  Airport Terminal  American Restaurant  Antique Shop  \
0                0                 0                    0             0   
1                0                 0                    0             0   
2                0                 0                    0             0   
3                0                 0                    0             0   
4                0                 0                    0             0   

   Arcade  Art Gallery  Art Museum  Arts & Crafts Store  Asian Restaurant  \
0       0            0           0                    0                 0   
1       0            0           0                    0                 0   
2       0            0           0                    0                 0   
3       0            0           0                    0                 0   
4       0            0           0                    0                 0   

   Athletics & Sports  Australian Restaurant  Auto Workshop  BBQ Joint  \
0                   0                      0              0          0   
1                   0                      0              0          0   
2                   0                      0              0          0   
3                   0                      0              0          0   
4                   0                      0              0          0   

   Bagel Shop  Bakery  Bank  Bar  Basketball Court  Bed & Breakfast  \
0           0       0     0    0                 0                0   
1           0       0     0    0                 0                0   
2           0       0     0    0                 0                0   
3           0       0     0    0                 0                0   
4           0       0     0    0                 0                0   

   Beer Garden  Bengali Restaurant  Big Box Store  Bistro  Boat or Ferry  \
0            0                   0              0       0              0   
1            0                   0              0       0              0   
2            0                   0              0       0              0   
3            0                   0              0       0              0   
4            0                   0              0       0              0   

   Bookstore  Boutique  Bowling Alley  Breakfast Spot  Brewery  Buffet  \
0          0         0              0               0        0       0   
1          0         0              0               0        0       0   
2          0         0              0               0        0       0   
3          0         0              0               0        0       0   
4          0         0              0               0        0       0   

   Building  Burger Joint  Burmese Restaurant  Bus Station  Business Service  \
0         0             0                   0            0                 0   
1         0             0                   0            0                 0   
2         0             0                   0            0                 0   
3         0             0                   0            0                 0   
4         0             0                   0            0                 0   

   Cafeteria  Café  Castle  Cheese Shop  Chinese Restaurant  Clothing Store  \
0          0     0       0            0                   0               0   
1          0     1       0            0                   0               0   
2          0     0       0            0                   0               0   
3          0     0       0            0                   0               0   
4    

In [23]:
nearby_onehot.shape

(5160, 213)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [24]:
nearby_grouped = nearby_onehot.groupby('Neighbourhood').mean().reset_index()
nearby_grouped

Neighbourhood       ATM  Accessories Store   Airport  \
0          Adarsh Nagar  0.000000              0.000  0.000000   
1             Alaknanda  0.000000              0.000  0.000000   
2           Anand Vihar  0.000000              0.000  0.000000   
3           Ashok Nagar  0.000000              0.000  0.000000   
4           Ashok Vihar  0.000000              0.000  0.000000   
5              Badarpur  0.000000              0.000  0.000000   
6            Bali Nagar  0.000000              0.000  0.000000   
7       Barakhamba Road  0.000000              0.000  0.000000   
8              BegumPur  0.000000              0.000  0.000000   
9          Chanakyapuri  0.000000              0.000  0.000000   
10        Chandni Chowk  0.000000              0.000  0.000000   
11    Chittaranjan Park  0.000000              0.000  0.000000   
12          Civil Lines  0.000000              0.000  0.000000   
13      Connaught Place  0.000000              0.000  0.000000   
14     Dabri, New Delhi  0.000000              0.000  0.000000   
15            Daryaganj  0.000000              0.000  0.000000   
16       Defence Colony  0.000000              0.000  0.000000   
17     Delhi Cantonment  0.000000              0.000  0.000000   
18          Dhaula Kuan  0.000000              0.000  0.000000   
19       Dilshad Garden  0.000000              0.000  0.000000   
20               Dwarka  0.000000              0.000  0.000000   
21     East Vinod Nagar  0.000000              0.000  0.000000   
22      East of Kailash  0.000000              0.000  0.000000   
23          Fateh Nagar  0.000000              0.000  0.000000   
24          Gole Market  0.000000              0.000  0.000000   
25           Govindpuri  0.000000              0.000  0.000000   
26      Greater Kailash  0.000000              0.000  0.000000   
27           Green Park  0.000000              0.000  0.000000   
28          Gulabi Bagh  0.000000              0.000  0.000000   
29        Gulmohar Park  0.000000              0.000  0.000000   
30            Hauz Khas  0.000000              0.000  0.000000   
31    Hauz Khas Village  0.000000              0.000  0.000000   
32           INA Colony  0.000000              0.000  0.000000   
33            Inderpuri  0.000000              0.000  0.000000   
34             Jangpura  0.000000              0.000  0.000000   
35          Jhandewalan  0.000000              0.000  0.000000   
36       Jhilmil Colony  0.000000              0.000  0.000000   
37             Jor Bagh  0.000000              0.000  0.000000   
38              Kalkaji  0.000000              0.000  0.000000   
39          Kamla Nagar  0.000000              0.000  0.000000   
40               Karala  0.000000              0.000  0.000000   
41           Karol Bagh  0.000000              0.000  0.000000   
42        Kashmiri Gate  0.000000              0.000  0.000000   
43              Khanpur  0.000000              0.000  0.000000   
44          Kirti Nagar  0.000000              0.000  0.000000   
45           Kishanganj  0.000000              0.000  0.000000   
46              Kotwali  0.000000              0.000  0.000000   
47         Lajpat Nagar  0.000000              0.000  0.000000   
48          Laxmi Nagar  0.000000              0.000  0.000000   
49       Laxmibai Nagar  0.000000              0.000  0.000000   
50          Lodi Colony  0.000000              0.000  0.000000   
51           Mahipalpur  0.000000              0.025  0.000000   
52        Malviya Nagar  0.000000              0.000  0.000000   
53          Mayur Vihar  0.000000              0.000  0.000000   
54             Mehrauli  0.000000              0.000  0.000000   
55           Model Town  0.000000              0.000  0.000000   
56     Mori Gate; Delhi  0.000000              0.000  0.000000   
57            Moti Bagh  0.000000              0.000  0.000000   
58           Moti Nagar  0.000000              0.000  0.000000   
59              Munirka  0.000000              0.0

#### Shape of this new dataframe nearby_grouped

In [25]:
nearby_grouped.shape

(117, 213)

#### Selecting column of Interest

As we are interseted to know the area where we can open bar in Delhi. So we select only those columns which are relevant for our analysis.

In [26]:
dl_bar = nearby_grouped[["Neighbourhood","Bar"]]
dl_bar.head()

Neighbourhood  Bar
0  Adarsh Nagar  0.0
1     Alaknanda  0.0
2   Anand Vihar  0.0
3   Ashok Nagar  0.0
4   Ashok Vihar  0.0

## 5. Clusters Neighbourhoods

In [27]:
#Run k-means to cluster the neighborhood into 3 clusters.

# set number of clusters
kclusters = 3

dl_bar_clustering = dl_bar.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dl_bar_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 0, 2, 2, 0])

#### Adding new column to the database dl_bar

In [28]:
dl_bar["Cluster_Label"]=kmeans.labels_
dl_bar.head()

<ipython-input-28-8054a6240e9b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dl_bar["Cluster_Label"]=kmeans.labels_


Neighbourhood  Bar  Cluster_Label
0  Adarsh Nagar  0.0              1
1     Alaknanda  0.0              1
2   Anand Vihar  0.0              1
3   Ashok Nagar  0.0              1
4   Ashok Vihar  0.0              1

#### Let's create a new dataframe that includes the cluster as well as geograohical coordinates of venues for each neighborhood.

In [29]:
dl_merged = df.join(dl_bar.set_index('Neighbourhood'), on='Neighbourhood')
dl_merged.head() 

Borough Neighbourhood   Latitude  Longitude   Bar  Cluster_Label
0   North West Delhi  Adarsh Nagar  28.614193  77.071541  0.00              1
1   North West Delhi   Ashok Vihar  28.699453  77.184826  0.00              1
2   North West Delhi      BegumPur  28.536978  77.209941  0.05              2
3   North West Delhi        Karala  28.735140  77.032511  0.00              1
4   North West Delhi    Model Town  28.702714  77.193991  0.00              1

#### Creating map of clusters

In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dl_merged['Latitude'], dl_merged['Longitude'], dl_merged['Neighbourhood'], dl_merged['Cluster_Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The result from the clustering are visualized in the map above with cluster 0 in red color, cluster 1 in violet color and cluster 2 in aquamarine color.

#### Saving map

In [32]:
map_clusters.save("map_clusters.html")

## 6. Examine Clusters

#### Types of cluster

In [33]:
dl_merged['Cluster_Label'].unique()

array([1, 2, 0])

#### Analysing Cluster 0

In [34]:
dl_merged[dl_merged['Cluster_Label']==0]

Borough       Neighbourhood   Latitude  Longitude       Bar  \
13         North Delhi             Kotwali  28.651718  77.221939  0.023256   
21         North Delhi           Wazirabad  28.433762  77.087757  0.019231   
30       Central Delhi         Jhandewalan  28.644319  77.199917  0.014706   
31       Central Delhi          Karol Bagh  28.652998  77.189023  0.024390   
34       Central Delhi      Rajinder Nagar  28.635786  77.184140  0.019608   
36           New Delhi        Chanakyapuri  28.594677  77.188521  0.013889   
39           New Delhi          INA Colony  28.577281  77.212649  0.014706   
53         South Delhi   Chittaranjan Park  28.538752  77.249249  0.033708   
55         South Delhi     East of Kailash  28.557032  77.244614  0.030000   
56         South Delhi          Govindpuri  28.535156  77.263794  0.029412   
57         South Delhi     Greater Kailash  28.559157  77.246018  0.030000   
64         South Delhi             Kalkaji  28.537070  77.261805  0.028571   
67         South Delhi         Lodi Colony  28.590702  77.220921  0.027778   
70         South Delhi          Neeti Bagh  28.559251  77.216166  0.034884   
76         South Delhi  Safdarjung Enclave  28.565642  77.193438  0.030000   
85         South Delhi        Sriniwaspuri  28.565220  77.254704  0.031250   
92    South West Delhi         Dhaula Kuan  28.591891  77.161703  0.023256   
102   South West Delhi         Vasant Kunj  28.529249  77.154134  0.016949   
105         West Delhi          Bali Nagar  28.654138  77.128178  0.020833   
111         West Delhi        Punjabi Bagh  28.668945  77.132461  0.028571   

     Cluster_Label  
13               0  
21               0  
30               0  
31               0  
34               0  
36               0  
39               0  
53               0  
55               0  
56               0  
57               0  
64               0  
67               0  
70               0  
76               0  
85               0  
92               0  
102              0  
105              0  
111              0

#### Analysing Cluster 1

In [35]:
dl_merged[dl_merged['Cluster_Label']==1]

Borough       Neighbourhood   Latitude  Longitude       Bar  \
0     North West Delhi        Adarsh Nagar  28.614193  77.071541  0.000000   
1     North West Delhi         Ashok Vihar  28.699453  77.184826  0.000000   
3     North West Delhi              Karala  28.735140  77.032511  0.000000   
4     North West Delhi          Model Town  28.702714  77.193991  0.000000   
5     North West Delhi              Narela  28.842610  77.091835  0.000000   
6     North West Delhi          Pitam Pura  28.703268  77.132250  0.000000   
7     North West Delhi              Rohini  28.716209  77.117074  0.000000   
8     North West Delhi       Shalimar Bagh  28.717453  77.150867  0.000000   
9          North Delhi         Civil Lines  28.676851  77.225030  0.000000   
10         North Delhi         Gulabi Bagh  28.669649  77.194726  0.000000   
11         North Delhi         Kamla Nagar  28.680344  77.202129  0.000000   
12         North Delhi       Kashmiri Gate  28.669977  77.232059  0.000000   
14         North Delhi    Mori Gate; Delhi  28.664203  77.223739  0.000000   
15         North Delhi        Sadar Bazaar  28.577151  77.111153  0.000000   
16         North Delhi       Sarai Rohilla  28.667873  77.190267  0.000000   
17         North Delhi        Shakti Nagar  28.679790  77.194914  0.000000   
18         North Delhi      Shastri Nagar.  28.670088  77.181859  0.000000   
19         North Delhi          Tis Hazari  28.667163  77.216631  0.000000   
20         North Delhi            Timarpur  28.701263  77.218680  0.000000   
22    North East Delhi      Dilshad Garden  28.675826  77.321516  0.000000   
23    North East Delhi     Naveen Shahdara  28.677339  77.286016  0.000000   
24    North East Delhi        New Usmanpur  28.683855  77.256005  0.000000   
25    North East Delhi            Shahdara  28.673333  77.289025  0.000000   
26    North East Delhi        Shastri Park  28.668434  77.250530  0.000000   
27    North East Delhi        Yamuna Vihar  28.700372  77.272773  0.000000   
28       Central Delhi       Chandni Chowk  28.655983  77.232194  0.000000   
29       Central Delhi           Daryaganj  28.646090  77.243048  0.000000   
32       Central Delhi          Kishanganj  28.662744  77.197908  0.000000   
40           New Delhi      Laxmibai Nagar  28.575419  77.205109  0.000000   
41           New Delhi      Pragati Maidan  28.623475  77.242528  0.000000   
42          East Delhi    East Vinod Nagar  28.622766  77.307503  0.000000   
43          East Delhi      Jhilmil Colony  28.669814  77.307283  0.000000   
44          East Delhi         Laxmi Nagar  28.630553  77.277575  0.000000   
45          East Delhi         Mayur Vihar  28.613107  77.295722  0.000000   
46          East Delhi        Pandav Nagar  28.650024  77.153676  0.000000   
47          East Delhi         Preet Vihar  28.641441  77.295259  0.000000   
48          East Delhi         Anand Vihar  28.625691  77.101941  0.000000   
49          East Delhi  Vasundhara Enclave  28.601726  77.321122  0.000000   
50          East Delhi       Vishwas Nagar  28.664436  77.294960  0.000000   
51          East Delhi         Vivek Vihar  28.669164  77.312267  0.000000   
52         South Delhi           Alaknanda  28.529336  77.251632  0.000000   
54         South Delhi      Defence Colony  28.571222  77.231776  0.010638   
62         South Delhi            Jangpura  28.582457  77.241500  0.011494   
63         South Delhi            Jor Bagh  28.676608  77.158692  0.000000   
65         South Delhi             Khanpur  28.512798  77.232395  0.000000   
66         South Delhi        Lajpat Nagar  28.579262  77.244033  0.010870   
72         South Delhi        Netaji Nagar  28.575582  77.186163  0.000000   
73         South Delhi  New Friends Colony  28.567101  77.269764  0.000000   
74         South Delhi     Nizamuddin West  28.588365  77.244955  0.000000   
75         South Delhi               Okhla  28.563662  77.289055  0.000000   
77         South Delhi         Sainik

#### Analysing Cluster 2

In [36]:
dl_merged[dl_merged['Cluster_Label']==2]

Borough      Neighbourhood   Latitude  Longitude       Bar  \
2    North West Delhi           BegumPur  28.536978  77.209941  0.050000   
33      Central Delhi          Paharganj  28.641499  77.214061  0.060000   
35          New Delhi    Barakhamba Road  28.626952  77.229950  0.050000   
37          New Delhi    Connaught Place  28.631383  77.219792  0.050000   
38          New Delhi        Gole Market  28.633719  77.205627  0.060000   
58        South Delhi         Green Park  28.555537  77.202497  0.050000   
59        South Delhi      Gulmohar Park  28.557101  77.213005  0.040000   
60        South Delhi          Hauz Khas  28.544256  77.206707  0.040000   
61        South Delhi  Hauz Khas Village  28.553855  77.194713  0.050000   
68        South Delhi      Malviya Nagar  28.533920  77.212447  0.050000   
69        South Delhi           Mehrauli  28.521826  77.178323  0.090909   
71        South Delhi        Nehru Place  28.549257  77.252952  0.040816   
78        South Delhi              Saket  28.524411  77.213725  0.050000   
82        South Delhi  Sarvodaya Enclave  28.537478  77.202089  0.040000   
83        South Delhi          Siri Fort  28.552146  77.224698  0.060000   
84        South Delhi    South Extension  28.568715  77.216896  0.041096   

    Cluster_Label  
2               2  
33              2  
35              2  
37              2  
38              2  
58              2  
59              2  
60              2  
61              2  
68              2  
69              2  
71              2  
78              2  
82              2  
83              2  
84              2

## 7. Identifying cluster of Interest

We can draw following conclusions about clusters from our analysis by exploring the neighbourhoods of Delhi:-

```Cluster 0```: Neighbourhoods with moderate concentration of Bar

```Cluster 1```: Neighbourhoods with non-existence of Bar

```Cluster 2```: Neighbourhoods with higher concentration of Bar

As stated above,it's clearly visible that there is great opportunity and high potential areas of opening bar in ```cluster 1``` neighbourhoods which covers mainly West Delhi,East Delhi and North Delhi as there is no competition due to non-existence of bar there. If we talk about ```cluster 2``` neighbourhoods which covers mainly South Delhi and New Delhi which have high concentration of bar, bars are more likely to suffer from high competition due to oversupply of products. Meanwhile one can open bar in ```cluster 0``` which covers mainly some parts of South Delhi and Central Delhi who are able to stand out in moderate competitions by proving better services.

Therefore this project recommends that if business people are thinking to open bar in Neighbourhoods of Delhi,they should focus more on ```cluster 1``` neighbourhoods with no competition. If business people have good sales and marketing strategy,they can open bar in ```cluster 0``` neighbourhoods with moderate competition. Last but not the least, they should avoid considering ```cluster 2```which already have high concentration of Bar.